In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1_yOuaRupWcvvBB5tNnjVrtDllXqg6x4Q", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/03_00_intro.mp3"))


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Why Matter Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_02_why_matter_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


# Multi-Head Attention and Positional Encoding from Scratch

*Part 3 of the Vizuara series on Self-Attention from First Principles*
*Estimated time: 50 minutes*

In [ ]:
#@title 🎧 Listen: Why Matter Pe
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_03_why_matter_pe.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 1. Why Does This Matter?

In the previous notebook, we built scaled dot-product attention — a mechanism that lets every word attend to every other word. But a single attention head can only learn **one** pattern. What if the word "it" needs to simultaneously attend to "animal" (coreference), "was" (syntax), and "tired" (semantics)?

This is where **Multi-Head Attention** comes in: we run multiple attention operations in parallel, each with its own learned weight matrices, so the model can capture multiple types of relationships simultaneously.

There is also a subtle problem we have been ignoring: self-attention has **no concept of word order**. The mechanism treats "dog bites man" and "man bites dog" identically. **Positional Encoding** fixes this by injecting position information into the embeddings.

By the end of this notebook, you will:
- Implement Multi-Head Attention from scratch
- Visualize what different heads learn (they specialize!)
- Build sinusoidal positional encodings and understand their frequency structure
- See how position information changes attention patterns

In [ ]:
#@title 🎧 Listen: Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_04_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 2. Building Intuition

### Multi-Head Attention

Imagine you are an editor reviewing a manuscript. You need to check for:
- **Grammar:** Does the subject agree with the verb?
- **Coherence:** Do pronouns refer to the right nouns?
- **Flow:** Does each paragraph connect logically to the next?

No single pass can catch everything. You need to read the text multiple times, each time focusing on a different aspect. Multi-Head Attention does exactly this — it reads the sentence multiple times in parallel, each "head" looking for a different type of relationship.

### Positional Encoding

Now imagine reading the same manuscript, but someone has shuffled all the words. Could you still understand it? Of course not — word order matters enormously. Self-attention, by itself, is like reading a bag of shuffled words. Positional encoding is the page numbering that tells the model where each word belongs.

### Think About This

If we had infinite computational budget, would we prefer more heads with smaller dimensions, or fewer heads with larger dimensions? What are the trade-offs?

In [ ]:
#@title 🎧 Listen: Math
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_05_math.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Math Pe
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_06_math_pe.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 3. The Mathematics

### Multi-Head Attention

Given $h$ attention heads, the multi-head attention mechanism works as follows:

$$\text{head}_i = \text{Attention}(X W_i^Q, \; X W_i^K, \; X W_i^V)$$

$$\text{MultiHead}(X) = \text{Concat}(\text{head}_1, \ldots, \text{head}_h) \, W^O$$

**What this says computationally:** Split the model dimension $d_{\text{model}}$ into $h$ heads, each operating on $d_k = d_{\text{model}} / h$ dimensions. Each head has its own $W_i^Q, W_i^K, W_i^V$ matrices. Run attention independently in each head, concatenate the results, and project back to $d_{\text{model}}$ dimensions with $W^O$.

### Sinusoidal Positional Encoding

$$PE_{(pos, 2i)} = \sin\!\left(\frac{pos}{10000^{2i/d_{\text{model}}}}\right)$$
$$PE_{(pos, 2i+1)} = \cos\!\left(\frac{pos}{10000^{2i/d_{\text{model}}}}\right)$$

**What this says computationally:** For each position $pos$ and each dimension index $i$, compute a sinusoidal value. Even dimensions use sine, odd dimensions use cosine. The frequency decreases as $i$ increases — early dimensions oscillate rapidly (capturing fine position differences) while later dimensions oscillate slowly (capturing coarse position differences).

In [ ]:
#@title 🎧 Code Walkthrough: Single Head Recap
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_07_single_head_recap.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 4. Let's Build It — Component by Component

### 4.1 Single-Head Attention (Recap)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt
import numpy as np

def single_head_attention(Q, K, V, mask=None):
    """Scaled dot-product attention for a single head."""
    d_k = Q.shape[-1]
    scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(~mask, float('-inf'))
    weights = F.softmax(scores, dim=-1)
    output = torch.matmul(weights, V)
    return output, weights

# Quick test
Q_test = torch.randn(3, 4)
K_test = torch.randn(3, 4)
V_test = torch.randn(3, 4)
out, w = single_head_attention(Q_test, K_test, V_test)
print(f"Single head output shape: {out.shape}")
print(f"Attention weights shape: {w.shape}")

In [ ]:
#@title 🎧 Code Walkthrough: Mha Class
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_08_mha_class.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 Multi-Head Attention — The Reshaping Trick

The key insight in multi-head attention is that instead of creating separate weight matrices for each head, we use a single large weight matrix and **reshape** the output into multiple heads. This is mathematically equivalent but much more efficient on GPUs.

In [ ]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        # Single large projections (equivalent to separate per-head projections)
        self.W_q = nn.Linear(d_model, d_model, bias=False)
        self.W_k = nn.Linear(d_model, d_model, bias=False)
        self.W_v = nn.Linear(d_model, d_model, bias=False)
        self.W_o = nn.Linear(d_model, d_model, bias=False)

    def forward(self, x, mask=None):
        batch_size, seq_len, _ = x.size()

        # Step 1: Project to Q, K, V (all heads at once)
        Q = self.W_q(x)  # (batch, seq, d_model)
        K = self.W_k(x)
        V = self.W_v(x)

        # Step 2: Reshape into multiple heads
        # (batch, seq, d_model) -> (batch, seq, num_heads, d_k) -> (batch, num_heads, seq, d_k)
        Q = Q.view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        K = K.view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        V = V.view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)

        # Step 3: Scaled dot-product attention (per head)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(~mask, float('-inf'))
        attention_weights = F.softmax(scores, dim=-1)
        context = torch.matmul(attention_weights, V)

        # Step 4: Concatenate heads
        # (batch, num_heads, seq, d_k) -> (batch, seq, num_heads, d_k) -> (batch, seq, d_model)
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)

        # Step 5: Output projection
        output = self.W_o(context)

        return output, attention_weights

# Create and test
torch.manual_seed(42)
d_model = 32
num_heads = 4
mha = MultiHeadSelfAttention(d_model, num_heads)

x = torch.randn(1, 6, d_model)  # batch=1, seq=6, d_model=32
output, attn_weights = mha(x)

print(f"Input shape:            {x.shape}")
print(f"Output shape:           {output.shape}")
print(f"Attention weights shape: {attn_weights.shape}")
print(f"  -> {num_heads} heads, each with a 6x6 attention matrix")

In [ ]:
#@title 🎧 What to Look For: Mha Viz Random
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_09_mha_viz_random.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualize attention patterns from each head
fig, axes = plt.subplots(1, num_heads, figsize=(4*num_heads, 4))
words_demo = ['The', 'cat', 'sat', 'on', 'the', 'mat']

for head_idx in range(num_heads):
    ax = axes[head_idx]
    weights_np = attn_weights[0, head_idx].detach().numpy()
    im = ax.imshow(weights_np, cmap='Blues', vmin=0, vmax=weights_np.max())
    ax.set_xticks(range(6))
    ax.set_xticklabels(words_demo, rotation=45, fontsize=9)
    ax.set_yticks(range(6))
    ax.set_yticklabels(words_demo, fontsize=9)
    ax.set_title(f'Head {head_idx+1}', fontsize=12, fontweight='bold')

plt.suptitle('Attention Patterns Per Head (Random Init)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()
print("These are random patterns because the model is untrained.")
print("After training, each head would specialize in a different relationship type.")

In [ ]:
#@title 🎧 Code Walkthrough: Reshape Viz Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_10_reshape_viz_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.3 The Reshaping Step — Visualized

In [ ]:
# Let's trace the exact reshaping that happens
batch_size, seq_len = 1, 4
d_model_demo = 8
num_heads_demo = 2
d_k_demo = d_model_demo // num_heads_demo  # = 4

# Start with a single projected Q (before reshape)
Q_flat = torch.arange(1, batch_size * seq_len * d_model_demo + 1, dtype=torch.float32)
Q_flat = Q_flat.view(batch_size, seq_len, d_model_demo)

print("Q before reshape (batch=1, seq=4, d_model=8):")
print(Q_flat[0])

# Step 1: View into (batch, seq, num_heads, d_k)
Q_reshaped = Q_flat.view(batch_size, seq_len, num_heads_demo, d_k_demo)
print(f"\nAfter view (batch=1, seq=4, heads=2, d_k=4):")
for h in range(num_heads_demo):
    print(f"  Head {h}: {Q_reshaped[0, :, h, :].tolist()}")

# Step 2: Transpose to (batch, num_heads, seq, d_k)
Q_heads = Q_reshaped.transpose(1, 2)
print(f"\nAfter transpose (batch=1, heads=2, seq=4, d_k=4):")
for h in range(num_heads_demo):
    print(f"  Head {h}:")
    for s in range(seq_len):
        print(f"    Position {s}: {Q_heads[0, h, s, :].tolist()}")

In [ ]:
#@title 🎧 What to Look For: Reshape Viz Plot
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_11_reshape_viz_plot.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visual diagram of the reshaping
fig, axes = plt.subplots(1, 3, figsize=(18, 4))

# Before reshape
ax = axes[0]
data = Q_flat[0].numpy()
im = ax.imshow(data, cmap='viridis', aspect='auto')
ax.set_title('Before Reshape\n(seq=4, d_model=8)', fontsize=11, fontweight='bold')
ax.set_xlabel('d_model dimension')
ax.set_ylabel('Sequence position')
ax.set_xticks(range(8))
ax.set_yticks(range(4))

# Head 0
ax = axes[1]
data_h0 = Q_heads[0, 0].numpy()
im = ax.imshow(data_h0, cmap='viridis', aspect='auto', vmin=1, vmax=32)
ax.set_title('Head 0\n(seq=4, d_k=4)', fontsize=11, fontweight='bold')
ax.set_xlabel('d_k dimension')
ax.set_ylabel('Sequence position')
ax.set_xticks(range(4))
ax.set_yticks(range(4))

# Head 1
ax = axes[2]
data_h1 = Q_heads[0, 1].numpy()
im = ax.imshow(data_h1, cmap='viridis', aspect='auto', vmin=1, vmax=32)
ax.set_title('Head 1\n(seq=4, d_k=4)', fontsize=11, fontweight='bold')
ax.set_xlabel('d_k dimension')
ax.set_ylabel('Sequence position')
ax.set_xticks(range(4))
ax.set_yticks(range(4))

plt.suptitle('Multi-Head Reshaping: Splitting d_model into h Heads', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Code Walkthrough: Sinusoidal Pe Class
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_12_sinusoidal_pe_class.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.4 Sinusoidal Positional Encoding

In [ ]:
class SinusoidalPositionalEncoding(nn.Module):
    """
    Sinusoidal positional encoding as described in 'Attention Is All You Need'.
    """
    def __init__(self, d_model, max_len=5000):
        super().__init__()

        # Create a matrix of shape (max_len, d_model)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)  # (max_len, 1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2, dtype=torch.float32) * (-math.log(10000.0) / d_model)
        )  # (d_model/2,)

        pe[:, 0::2] = torch.sin(position * div_term)  # Even dimensions
        pe[:, 1::2] = torch.cos(position * div_term)  # Odd dimensions

        # Register as a buffer (not a parameter — not trained)
        self.register_buffer('pe', pe.unsqueeze(0))  # (1, max_len, d_model)

    def forward(self, x):
        """Add positional encoding to input embeddings."""
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len, :]

# Create and inspect
pe_module = SinusoidalPositionalEncoding(d_model=64, max_len=100)
print(f"PE buffer shape: {pe_module.pe.shape}")

# Show the PE values for position 0 and position 3
print(f"\nPE[0, :4] = {pe_module.pe[0, 0, :4].tolist()}")
print(f"PE[3, :4] = {pe_module.pe[0, 3, :4].tolist()}")

In [ ]:
#@title 🎧 Code Walkthrough: Sinusoidal Pe Verify
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_13_sinusoidal_pe_verify.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let us verify the worked example from the article.

In [ ]:
# Verify the hand-computed PE from the article
# PE at position 3 with d_model=4
d_model_small = 4
pe_small = SinusoidalPositionalEncoding(d_model_small, max_len=10)

pe_pos3 = pe_small.pe[0, 3, :].tolist()
print("Positional encoding for position 3 (d_model=4):")
print(f"  PE[3] = {[round(v, 3) for v in pe_pos3]}")
print(f"\nExpected from article:")
print(f"  PE[3] = [sin(3), cos(3), sin(0.03), cos(0.03)]")
print(f"        = [{math.sin(3):.3f}, {math.cos(3):.3f}, {math.sin(0.03):.3f}, {math.cos(0.03):.3f}]")

In [ ]:
#@title 🎧 What to Look For: Sinusoidal Pe Heatmap
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_14_sinusoidal_pe_heatmap.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualize the positional encoding heatmap
fig, ax = plt.subplots(figsize=(12, 6))

pe_data = pe_module.pe[0, :50, :64].numpy()  # First 50 positions, first 64 dims
im = ax.imshow(pe_data, cmap='RdBu_r', aspect='auto', vmin=-1, vmax=1)
ax.set_xlabel('Dimension Index', fontsize=12)
ax.set_ylabel('Position', fontsize=12)
ax.set_title('Sinusoidal Positional Encoding', fontsize=14, fontweight='bold')
plt.colorbar(im, ax=ax, label='Encoding Value')
plt.tight_layout()
plt.show()

print("Left side: rapidly oscillating (fine-grained position)")
print("Right side: slowly oscillating (coarse position)")

In [ ]:
#@title 🎧 What to Look For: Sinusoidal Pe Frequencies
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_15_sinusoidal_pe_frequencies.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Key property: relative positions can be represented as linear transformations
# Let's show that PE(pos+k) can be expressed as a rotation of PE(pos)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

positions = list(range(20))
pe_values = pe_module.pe[0, :20, :].numpy()

# Plot sinusoidal waves at different frequencies
for ax, dim_pair, title in zip(axes,
                                [(0, 1), (10, 11), (30, 31)],
                                ['Dimensions 0-1\n(High Frequency)',
                                 'Dimensions 10-11\n(Medium Frequency)',
                                 'Dimensions 30-31\n(Low Frequency)']):
    ax.plot(positions, pe_values[:, dim_pair[0]], 'o-', label=f'sin (dim {dim_pair[0]})',
            color='#3498db', linewidth=2, markersize=4)
    ax.plot(positions, pe_values[:, dim_pair[1]], 's-', label=f'cos (dim {dim_pair[1]})',
            color='#e74c3c', linewidth=2, markersize=4)
    ax.set_xlabel('Position', fontsize=11)
    ax.set_ylabel('Value', fontsize=11)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-1.2, 1.2)

plt.suptitle('Sinusoidal Waves at Different Frequencies', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Code Walkthrough: Pe Affect Attn Setup
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_16_pe_affect_attn_setup.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.5 How Positional Encoding Affects Attention

In [ ]:
#@title 🎧 What to Look For: Pe Affect Attn Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_17_pe_affect_attn_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Demonstrate that positional encoding changes attention patterns
torch.manual_seed(42)

d_model_test = 32
seq_len_test = 8
num_heads_test = 4

# Create identical embeddings for all positions (same word repeated)
# Without PE: attention should be uniform
# With PE: attention should show positional bias
embedding_test = nn.Embedding(1, d_model_test)
same_word_input = torch.zeros(1, seq_len_test, dtype=torch.long)  # all same word
x_no_pe = embedding_test(same_word_input)  # (1, 8, 32)

# Create PE
pe_test = SinusoidalPositionalEncoding(d_model_test)
x_with_pe = pe_test(x_no_pe.clone())

# Run through MHA
mha_test = MultiHeadSelfAttention(d_model_test, num_heads_test)

_, attn_no_pe = mha_test(x_no_pe)
_, attn_with_pe = mha_test(x_with_pe)

fig, axes = plt.subplots(2, num_heads_test, figsize=(4*num_heads_test, 8))

for h in range(num_heads_test):
    # Without PE
    ax = axes[0][h]
    im = ax.imshow(attn_no_pe[0, h].detach().numpy(), cmap='Blues', vmin=0)
    ax.set_title(f'Head {h+1}', fontsize=11, fontweight='bold')
    if h == 0:
        ax.set_ylabel('Without PE', fontsize=12, fontweight='bold')

    # With PE
    ax = axes[1][h]
    im = ax.imshow(attn_with_pe[0, h].detach().numpy(), cmap='Blues', vmin=0)
    ax.set_title(f'Head {h+1}', fontsize=11, fontweight='bold')
    if h == 0:
        ax.set_ylabel('With PE', fontsize=12, fontweight='bold')

plt.suptitle('Attention Patterns: Same Word at Every Position\n'
             'Without PE (top) vs With PE (bottom)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()
print("Without PE: attention is nearly uniform (no position info).")
print("With PE: each position attends differently based on relative distance.")

In [ ]:
#@title 🎧 Before You Start: Todo1 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_18_todo1_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 5. Your Turn

### TODO 1: Implement Multi-Head Attention Without the Reshaping Trick

In [ ]:
#@title 🎧 Before You Start: Todo1 Guidance
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_19_todo1_guidance.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class MultiHeadAttentionExplicit(nn.Module):
    """
    Multi-Head Attention implemented with explicit separate heads.
    This is less efficient but makes the logic crystal clear.

    Args:
        d_model: Total model dimension
        num_heads: Number of attention heads
    """
    def __init__(self, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        # ============ TODO ============
        # Create separate W_q, W_k, W_v for EACH head
        # Use nn.ModuleList to hold them
        # Each projects from d_model to d_k
        #
        # Also create the output projection W_o
        # from d_model to d_model
        # ==============================

        self.W_q_heads = ???  # YOUR CODE HERE: nn.ModuleList of nn.Linear
        self.W_k_heads = ???  # YOUR CODE HERE
        self.W_v_heads = ???  # YOUR CODE HERE
        self.W_o = ???        # YOUR CODE HERE

    def forward(self, x):
        """
        Args:
            x: Input of shape (batch_size, seq_len, d_model)
        Returns:
            output: Shape (batch_size, seq_len, d_model)
            attention_weights: Shape (batch_size, num_heads, seq_len, seq_len)
        """
        head_outputs = []
        all_weights = []

        # ============ TODO ============
        # For each head i:
        #   1. Compute Q_i = self.W_q_heads[i](x)
        #   2. Compute K_i = self.W_k_heads[i](x)
        #   3. Compute V_i = self.W_v_heads[i](x)
        #   4. Compute attention: scores = Q_i @ K_i.T / sqrt(d_k)
        #   5. Apply softmax
        #   6. Compute output = weights @ V_i
        #   7. Collect the output and weights
        #
        # After the loop:
        #   8. Concatenate all head outputs along the last dimension
        #   9. Apply W_o
        # ==============================

        for i in range(self.num_heads):
            pass  # YOUR CODE HERE

        # Concatenate and project
        output = ???  # YOUR CODE HERE
        attn = ???    # YOUR CODE HERE

        return output, attn

In [ ]:
#@title 🎧 Code Walkthrough: Todo1 Verify
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_20_todo1_verify.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Code Walkthrough: Todo2 Verify
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_23_todo2_verify.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Verification: compare explicit and efficient implementations
torch.manual_seed(42)
d_m = 16
n_h = 4
x_verify = torch.randn(2, 5, d_m)

mha_efficient = MultiHeadSelfAttention(d_m, n_h)
out_eff, w_eff = mha_efficient(x_verify)

# If you implemented the explicit version, test it:
# mha_explicit = MultiHeadAttentionExplicit(d_m, n_h)
# out_exp, w_exp = mha_explicit(x_verify)
# print(f"Efficient output shape: {out_eff.shape}")
# print(f"Explicit output shape: {out_exp.shape}")

print(f"Output shape: {out_eff.shape}")
assert out_eff.shape == (2, 5, d_m), f"Expected (2, 5, {d_m}), got {out_eff.shape}"
assert w_eff.shape == (2, n_h, 5, 5), f"Expected (2, {n_h}, 5, 5), got {w_eff.shape}"
print("Shape assertions passed!")

In [ ]:
#@title 🎧 Before You Start: Todo2 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_21_todo2_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### TODO 2: Implement Learned Positional Encoding

In [ ]:
#@title 🎧 Before You Start: Todo2 Guidance
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_22_todo2_guidance.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class LearnedPositionalEncoding(nn.Module):
    """
    Learned (trainable) positional encoding.
    Unlike sinusoidal, these are trained along with the model.
    Used in BERT and GPT-2.

    Args:
        d_model: Embedding dimension
        max_len: Maximum sequence length
    """
    def __init__(self, d_model, max_len=512):
        super().__init__()
        # ============ TODO ============
        # Create a learnable embedding table for positions
        # Hint: nn.Embedding(max_len, d_model)
        # ==============================

        self.position_embedding = ???  # YOUR CODE HERE

    def forward(self, x):
        """
        Args:
            x: Input of shape (batch_size, seq_len, d_model)
        Returns:
            x + positional encoding
        """
        seq_len = x.size(1)

        # ============ TODO ============
        # 1. Create position indices: [0, 1, 2, ..., seq_len-1]
        # 2. Look up the positional embeddings
        # 3. Add them to x
        # ==============================

        positions = ???  # YOUR CODE HERE
        pe = ???         # YOUR CODE HERE
        return x + pe

In [ ]:
# Verification
lpe = LearnedPositionalEncoding(d_model=32, max_len=100)
test_x = torch.randn(2, 10, 32)
out_lpe = lpe(test_x)

assert out_lpe.shape == test_x.shape, f"Shape mismatch: {out_lpe.shape} vs {test_x.shape}"
assert not torch.allclose(out_lpe, test_x), "Output should differ from input (PE added)"
print("Learned PE assertions passed!")

# Compare: sinusoidal is deterministic, learned is random at init
spe = SinusoidalPositionalEncoding(d_model=32)
out_spe = spe(test_x)
print(f"\nSinusoidal PE (deterministic): first 4 values = {out_spe[0, 0, :4].tolist()}")
print(f"Learned PE (random init): first 4 values = {out_lpe[0, 0, :4].tolist()}")

In [ ]:
#@title 🎧 Code Walkthrough: Putting Together
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_24_putting_together.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Putting It All Together

In [ ]:
class TransformerInput(nn.Module):
    """
    Full input processing pipeline for a Transformer:
    Embedding + Positional Encoding -> Multi-Head Self-Attention
    """
    def __init__(self, vocab_size, d_model, num_heads, max_len=512):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = SinusoidalPositionalEncoding(d_model, max_len)
        self.attention = MultiHeadSelfAttention(d_model, num_heads)
        self.d_model = d_model

    def forward(self, token_ids, mask=None):
        # Step 1: Token embedding
        x = self.embedding(token_ids) * math.sqrt(self.d_model)  # Scale by sqrt(d_model)

        # Step 2: Add positional encoding
        x = self.pos_encoding(x)

        # Step 3: Multi-Head Self-Attention
        output, attention_weights = self.attention(x, mask)

        return output, attention_weights

# Create and test
torch.manual_seed(42)
model = TransformerInput(vocab_size=100, d_model=32, num_heads=4)

# Fake token IDs for "The cat sat on the mat"
token_ids = torch.tensor([[1, 5, 23, 8, 1, 42]])  # batch=1, seq=6
output, attn_weights = model(token_ids)

print(f"Token IDs shape:        {token_ids.shape}")
print(f"Output shape:           {output.shape}")
print(f"Attention weights shape: {attn_weights.shape}")

In [ ]:
#@title 🎧 What to Look For: Combined Attn Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_25_combined_attn_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualize the complete pipeline
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
word_labels = ['The', 'cat', 'sat', 'on', 'the', 'mat']

for h in range(4):
    ax = axes[h // 2][h % 2]
    w = attn_weights[0, h].detach().numpy()
    im = ax.imshow(w, cmap='Blues', vmin=0, vmax=w.max())
    ax.set_xticks(range(6))
    ax.set_xticklabels(word_labels, rotation=45, fontsize=10)
    ax.set_yticks(range(6))
    ax.set_yticklabels(word_labels, fontsize=10)
    ax.set_title(f'Head {h+1}', fontsize=12, fontweight='bold')
    plt.colorbar(im, ax=ax, shrink=0.8)

plt.suptitle('Multi-Head Attention: "The cat sat on the mat"\n'
             '(With Positional Encoding, Random Init)',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Training Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_26_training_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. Training and Results

Let us train the model on a word-ordering task to see heads specialize.

In [ ]:
torch.manual_seed(42)

# Task: given a scrambled sentence, predict the correct order
# We train the attention to learn positional relationships

vocab = ['<pad>', 'the', 'cat', 'sat', 'on', 'mat', 'dog', 'ran', 'to', 'park']
word2idx = {w: i for i, w in enumerate(vocab)}
vocab_size = len(vocab)
d_model = 32
num_heads = 4

# Full model
model = TransformerInput(vocab_size, d_model, num_heads)
classifier = nn.Linear(d_model, vocab_size)

params = list(model.parameters()) + list(classifier.parameters())
optimizer = torch.optim.Adam(params, lr=0.005)

# Training sequences (predict next word)
sequences = [
    ([1, 2, 3, 4, 1], 5),    # the cat sat on the -> mat
    ([1, 6, 7, 8, 1], 9),    # the dog ran to the -> park
    ([4, 1, 5, 1, 2], 3),    # on the mat the cat -> sat
    ([8, 1, 9, 1, 6], 7),    # to the park the dog -> ran
]

losses = []

for epoch in range(400):
    total_loss = 0
    for seq, target in sequences:
        token_ids = torch.tensor([seq])
        out, _ = model(token_ids)
        logits = classifier(out[0, -1])  # Last position
        loss = F.cross_entropy(logits.unsqueeze(0), torch.tensor([target]))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    losses.append(total_loss / len(sequences))

print(f"Final loss: {losses[-1]:.4f}")

In [ ]:
#@title 🎧 What to Look For: Training Curve Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_27_training_curve_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Plot training curve
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(losses, color='#3498db', linewidth=2)
ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('Loss', fontsize=12)
ax.set_title('Training Multi-Head Attention for Next-Word Prediction', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Transition: Final Output Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_28_final_output_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Final Output

In [ ]:
#@title 🎧 What to Look For: Final Output Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_29_final_output_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualize the learned attention patterns after training
fig, axes = plt.subplots(len(sequences), num_heads, figsize=(4*num_heads, 4*len(sequences)))

with torch.no_grad():
    for s_idx, (seq, target) in enumerate(sequences):
        token_ids = torch.tensor([seq])
        _, attn = model(token_ids)
        seq_words = [vocab[i] for i in seq]

        for h in range(num_heads):
            ax = axes[s_idx][h]
            w = attn[0, h].numpy()
            im = ax.imshow(w, cmap='Blues', vmin=0, vmax=w.max())
            ax.set_xticks(range(5))
            ax.set_xticklabels(seq_words, rotation=45, fontsize=8)
            ax.set_yticks(range(5))
            ax.set_yticklabels(seq_words, fontsize=8)
            if s_idx == 0:
                ax.set_title(f'Head {h+1}', fontsize=11, fontweight='bold')
            if h == 0:
                ax.set_ylabel(f'{" ".join(seq_words)}\n->{vocab[target]}',
                            fontsize=9, fontweight='bold')

plt.suptitle('Learned Multi-Head Attention Patterns\n'
             'Each Head Specializes in Different Relationships',
             fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nCongratulations! You have built Multi-Head Attention and Positional Encoding from scratch!")
print("Notice how different heads have learned different attention patterns.")
print("Next up: assembling the full Transformer block with residual connections,")
print("layer normalization, and the feed-forward network.")

In [ ]:
#@title 🎧 Wrap-Up: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_30_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 9. Reflection and Next Steps

### Reflection Questions
1. Why is the standard practice to set $d_k = d_{\text{model}} / h$ rather than having each head use the full $d_{\text{model}}$ dimensions? What happens to computation cost if each head uses the full dimension?
2. Compare sinusoidal and learned positional encodings. What are the advantages of each? (Hint: think about generalization to longer sequences.)
3. In the trained attention patterns, do different heads appear to focus on different things? What patterns can you identify?

### Optional Challenges
1. Implement **Rotary Positional Encoding (RoPE)** — the positional encoding used in LLaMA and modern LLMs. It encodes relative position directly into the attention computation.
2. Try training with different numbers of heads (1, 2, 4, 8) and compare the final loss. Is there a sweet spot?